In [2]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

In [3]:
df_cleantech_epo = pd.read_csv('/mnt/hdd01/PATSTAT Working Directory/cleantech_ep_granted.csv')
df_rel_on_science = pd.read_csv('/mnt/hdd01/Reliance on Science/Raw Files/_pcs_oa.csv')

In [5]:
# Pad publn_nr to 7 digits, append 0 in front if necessary
df_cleantech_epo['publn_nr'] = df_cleantech_epo['publn_nr'].astype(str).str.zfill(7)

In [8]:
# Create new column patent_id in df_rel_on_science, extract number between two hyphens
df_rel_on_science['publn_nr'] = df_rel_on_science['patent'].str.extract(r'(\d+)-')
# Drop all rows in df_rel_science where patent does not contain 'ep'
df_rel_on_science = df_rel_on_science[df_rel_on_science['patent'].str.contains('ep')]

In [12]:
# Pad publn_nr to 7 digits, append 0 in front if necessary
df_rel_on_science['publn_nr'] = df_rel_on_science['publn_nr'].astype(str).str.zfill(7)

In [14]:
# Merge df_cleantech_epo and df_rel_on_science on publn_nr
df_cleantech_epo_rel_on_science = pd.merge(df_cleantech_epo, df_rel_on_science, on='publn_nr', how='inner', validate='many_to_many')
# Aggregate df_cleantech_epo_rel_on_science on oaid, list publn_nr, appln_title, cpc_class_symbol
df_cleantech_epo_rel_on_science_agg = df_cleantech_epo_rel_on_science.groupby('oaid').agg({'publn_nr': lambda x: list(x), 'appln_title': lambda x: list(x), 'cpc_class_symbol': lambda x: list(x)})

In [64]:
# Reset index
df_cleantech_epo_rel_on_science_agg.index.name = 'oaid'
df_cleantech_epo_rel_on_science_agg.reset_index()

# Reorder columns
df_cleantech_epo_rel_on_science_agg = df_cleantech_epo_rel_on_science_agg[['oaid', 'publn_nr', 'appln_title', 'cpc_class_symbol']]

In [66]:
df_cleantech_epo_rel_on_science_agg.to_json('/mnt/hdd01/PATSTAT Working Directory/Reliance on Science/cleantech_epo_rel_on_science_agg.json', orient='records')